# Lab 3: Mastering A2C (and Kung-Fu)

This part is based on [Practical RL week08 practice](https://github.com/yandexdataschool/Practical_RL/tree/master/week08_pomdp). All rights belong to original authors

In [ ]:
import os
import sys


if "google.colab" in sys.modules and not os.path.exists(".setup_complete"):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !wget -q https://raw.githubusercontent.com/girafe-ai/ml-mipt/develop_advanced_practice/homeworks_advanced/Lab2_RL/atari_util.py
    !wget -q https://raw.githubusercontent.com/girafe-ai/ml-mipt/develop_advanced_practice/homeworks_advanced/Lab2_RL/env_pool.py
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ["DISPLAY"] = ":1"

## Kung-Fu, recurrent style

In this notebook we'll once again train RL agent for for atari [KungFuMaster](https://gym.openai.com/envs/KungFuMaster-v0), this time using recurrent neural networks.

![https://www.atariage.com/2600/screenshots/s_KungFuMaster_2.png](https://www.atariage.com/2600/screenshots/s_KungFuMaster_2.png)

In [ ]:
import gym
from atari_util import PreprocessAtari


def make_env():
    env = gym.make("KungFuMasterDeterministic-v0")
    env = PreprocessAtari(
        env, height=42, width=42, crop=lambda img: img[60:-30, 15:], color=False, n_frames=1
    )
    return env


env = make_env()

obs_shape = env.observation_space.shape
n_actions = env.action_space.n

print("Observation shape:", obs_shape)
print("Num actions:", n_actions)
print("Action names:", env.env.env.get_action_meanings())

In [ ]:
import matplotlib.pyplot as plt


s = env.reset()
for _ in range(100):
    s, _, _, _ = env.step(env.action_space.sample())

plt.title('Game image')
plt.imshow(env.render('rgb_array'))
plt.show()

plt.title('Agent observation')
plt.imshow(s.reshape([42, 42]))
plt.show()

## POMDP setting

The atari game we're working with is actually a POMDP: your agent needs to know timing at which enemies spawn and move, but cannot do so unless it has some memory. 

Let's design another agent that has a recurrent neural net memory to solve this. Here's a sketch.

![img](https://github.com/girafe-ai/ml-mipt/blob/master/homeworks_advanced/Lab2_RL/img1.jpg?raw=1)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleRecurrentAgent(nn.Module):
    def __init__(self, obs_shape, n_actions, reuse=False):
        """A simple actor-critic agent"""
        super().__init__()

        self.obs_features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2),
            nn.ELU(),
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(),
        )

        self.rnn = nn.LSTMCell(128, 128)

        self.logits = nn.Linear(128, n_actions)
        self.state_value = nn.Linear(128, 1)

    def forward(self, prev_state, obs_t):
        """
        Takes agent's previous step and observation,
        returns next state and whatever it needs to learn (tf tensors)
        """

        # YOUR CODE HERE
        # apply the whole neural net for one step here.
        # See docs on self.rnn(...)
        # the recurrent cell should take the last feedforward dense layer as input
        # new_state = ...
        # logits = ...
        # state_value = ...

        return new_state, (logits, state_value)

    def get_initial_state(self, batch_size):
        """Return a list of agent memory states at game start. Each state is a np array of shape [batch_size, ...]"""
        device = next(self.parameters()).device
        return torch.zeros((batch_size, 128), device=device), torch.zeros(
            (batch_size, 128), device=device
        )

    def sample_actions(self, agent_outputs):
        """pick actions given numeric agent outputs (np arrays)"""
        logits, state_values = agent_outputs
        probs = F.softmax(logits, dim=1)
        return torch.multinomial(probs, 1)[:, 0].detach().cpu().numpy()

    def step(self, prev_state, obs_t):
        """ like forward, but obs_t is a numpy array """
        device = next(self.parameters()).device
        obs_t = torch.tensor(np.asarray(obs_t), dtype=torch.float32, device=device)
        (h, c), (l, s) = self(prev_state, obs_t)
        return (h.detach(), c.detach()), (l.detach(), s.detach())

In [ ]:
n_parallel_games = 10
gamma = 0.99

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
agent = SimpleRecurrentAgent(obs_shape, n_actions).to(device)

In [ ]:
state = [env.reset()]
_, (logits, value) = agent.step(agent.get_initial_state(1), state)
print("action logits:\n", logits)
print("state values:\n", value)

### Let's play!
Let's build a function that measures agent's average reward.

In [ ]:
def evaluate(agent, env, n_games=1):
    """Plays an entire game start to end, returns session rewards."""

    game_rewards = []
    for _ in range(n_games):
        # initial observation and memory
        observation = env.reset()
        prev_memories = agent.get_initial_state(1)

        total_reward = 0
        while True:
            new_memories, readouts = agent.step(prev_memories, observation[None, ...])
            action = agent.sample_actions(readouts)

            observation, reward, done, info = env.step(action[0])

            total_reward += reward
            prev_memories = new_memories
            if done:
                break

        game_rewards.append(total_reward)
    return game_rewards

In [ ]:
with gym.wrappers.Monitor(env, directory="videos", force=True) as env_monitor:
    rw = evaluate(agent, env_monitor, n_games=3)

print(rw)

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from base64 import b64encode
from pathlib import Path

from IPython.display import HTML


video_paths = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[-1]  # You can also try other indices

if 'google.colab' in sys.modules:
    # https://stackoverflow.com/a/57378660/1214547
    with video_path.open('rb') as fp:
        mp4 = fp.read()
    data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
else:
    data_url = str(video_path)

HTML(
    """
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(
        data_url
    )
)

### Training on parallel games

We introduce a class called EnvPool - it's a tool that handles multiple environments for you. Here's how it works:
![img](https://github.com/girafe-ai/ml-mipt/blob/master/homeworks_advanced/Lab2_RL/img2.jpg?raw=1)

In [ ]:
from env_pool import EnvPool


pool = EnvPool(agent, make_env, n_parallel_games)

We gonna train our agent on a thing called __rollouts:__
![img](https://github.com/girafe-ai/ml-mipt/blob/master/homeworks_advanced/Lab2_RL/img3.jpg?raw=1)

A rollout is just a sequence of T observations, actions and rewards that agent took consequently.
* First __s0__ is not necessarily initial state for the environment
* Final state is not necessarily terminal
* We sample several parallel rollouts for efficiency

In [ ]:
# for each of n_parallel_games, take 10 steps
rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(10)

In [ ]:
print("Actions shape:", rollout_actions.shape)
print("Rewards shape:", rollout_rewards.shape)
print("Mask shape:", rollout_mask.shape)
print("Observations shape: ", rollout_obs.shape)

# Actor-critic objective

Here we define a loss function that uses rollout above to train advantage actor-critic agent.


Our loss consists of three components:

* __The policy "loss"__
 $$ \hat J = {1 \over T} \cdot \sum_t { \log \pi(a_t | s_t) } \cdot A_{const}(s,a) $$
  * This function has no meaning in and of itself, but it was built such that
  * $ \nabla \hat J = {1 \over N} \cdot \sum_t { \nabla \log \pi(a_t | s_t) } \cdot A(s,a) \approx \nabla E_{s, a \sim \pi} R(s,a) $
  * Therefore if we __maximize__ J_hat with gradient descent we will maximize expected reward
  
  
* __The value "loss"__
  $$ L_{td} = {1 \over T} \cdot \sum_t { [r + \gamma \cdot V_{const}(s_{t+1}) - V(s_t)] ^ 2 }$$
  * Ye Olde TD_loss from q-learning and alike
  * If we minimize this loss, V(s) will converge to $V_\pi(s) = E_{a \sim \pi(a | s)} R(s,a) $


* __Entropy Regularizer__
  $$ H = - {1 \over T} \sum_t \sum_a {\pi(a|s_t) \cdot \log \pi (a|s_t)}$$
  * If we __maximize__ entropy we discourage agent from predicting zero probability to actions
  prematurely (a.k.a. exploration)
  
  
So we optimize a linear combination of $L_{td}$ $- \hat J$, $-H$

---

__One more thing:__ since we train on T-step rollouts, we can use N-step formula for advantage for free:
  * At the last step, $A(s_t,a_t) = r(s_t, a_t) + \gamma \cdot V(s_{t+1}) - V(s) $
  * One step earlier, $A(s_t,a_t) = r(s_t, a_t) + \gamma \cdot r(s_{t+1}, a_{t+1}) + \gamma ^ 2 \cdot V(s_{t+2}) - V(s) $
  * Et cetera, et cetera. This way agent starts training much faster since it's estimate of A(s,a) depends less on his (imperfect) value function and more on actual rewards. There's also a [nice generalization](https://arxiv.org/abs/1506.02438) of this.


__Note:__ it's also a good idea to scale rollout_len up to learn longer sequences. You may wish set it to >=20 or to start at 10 and then scale up as time passes.

In [ ]:
opt = torch.optim.Adam(agent.parameters(), lr=1e-5)


def train_on_rollout(
    states,
    actions,
    rewards,
    is_not_done,
    prev_memory_states,
    gamma=0.99,
    device=device,
    max_grad_norm=90,
):
    """
    Takes a sequence of states, actions and rewards produced by generate_session.
    Updates agent's weights by following the policy gradient above.
    Please use Adam optimizer with default parameters.
    """
    states = torch.tensor(
        np.asarray(states), dtype=torch.float32, device=device
    )  # shape: [batch_size, time, c, h, w]
    actions = torch.tensor(
        np.array(actions), dtype=torch.int64, device=device
    )  # shape: [batch_size, time]
    rewards = torch.tensor(
        np.array(rewards), dtype=torch.float32, device=device
    )  # shape: [batch_size, time]
    is_not_done = torch.tensor(
        np.array(is_not_done), dtype=torch.float32, device=device
    )  # shape: [batch_size, time]
    rollout_length = rewards.shape[1]

    # predict logits, probas and log-probas using an agent.
    memory = [m.detach() for m in prev_memory_states]

    logits = []  # append logit sequence here
    state_values = []  # append state values here
    for t in range(rollout_length):
        obs_t = states[:, t]

        # YOUR CODE HERE
        # use agent to comute logits_t and state values_t.
        # append them to logits and state_values array
        # memory, (logits_t, values_t) = ...

        logits.append(logits_t)
        state_values.append(values_t)

    logits = torch.stack(logits, dim=1)  # shape: [batch_size, time, n_actions]
    state_values = torch.stack(state_values, dim=1)  # shape: [batch_size, time, 1]
    probas = F.softmax(logits, dim=2)  # shape: [batch_size, time, n_actions]
    logprobas = F.log_softmax(logits, dim=2)  # shape: [batch_size, time, n_actions]

    # Torch's Linear layer outputs tensor with shape [batch_size, output_dim] even if output_dim equals to 1.
    # To keep state_values consistent with other tensors, we need to get rid of this last dimension.
    state_values = state_values.squeeze(dim=2)

    # select log-probabilities for chosen actions, log pi(a_i|s_i)
    logprobas_for_actions = torch.gather(logprobas, dim=2, index=actions[:, :, None])
    logprobas_for_actions = logprobas_for_actions.squeeze(dim=2)

    # Now let's compute two loss components:
    # 1) Policy gradient objective.
    # Notes: Please don't forget to call .detach() on advantage term. Also please use mean, not sum.
    # it's okay to use loops if you want
    J_hat = 0  # policy objective as in the formula for J_hat

    # 2) Temporal difference MSE for state values
    # Notes: Please don't forget to call on V(s') term. Also please use mean, not sum.
    # it's okay to use loops if you want
    value_loss = 0

    cumulative_returns = state_values[:, -1].detach()

    for t in reversed(range(rollout_length - 1)):
        r_t = rewards[:, t]  # current rewards
        V_t = state_values[:, t]  # current state values
        V_next = state_values[:, t + 1].detach()  # next state values
        logpi_a_s_t = logprobas_for_actions[:, t]  # log-probability of a_t in s_t

        # update G_t = r_t + gamma * G_{t+1} as we did in week6 reinforce
        cumulative_returns = G_t = r_t + gamma * cumulative_returns

        # YOUR CODE HERE
        # Compute temporal difference error (MSE for V(s))
        # value_loss += ...

        # YOUR CODE HERE
        # compute advantage A(s_t, a_t) using cumulative returns and V(s_t) as baseline
        # advantage = ...

        # We use advantage only as weights and as such we need to detach it from computation graph.
        advantage = advantage.detach()

        # YOUR CODE HERE
        # compute policy pseudo-loss aka -J_hat.
        # J_hat += ...

    # YOUR CODE HERE
    # regularize with entropy

    # add-up three loss components and average over time
    loss = -J_hat / rollout_length + value_loss / rollout_length + -0.01 * entropy_reg

    # Gradient descent step
    opt.zero_grad()
    loss.backward()

    # This small trick allows to clip gradients and to monitor them over the time
    grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
    opt.step()

    return loss.item(), grad_norm, entropy_reg.item()

In [ ]:
# let's test it
memory = list(pool.prev_memory_states)
rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(10)

_ = train_on_rollout(rollout_obs, rollout_actions, rollout_rewards, rollout_mask, memory)

# Train 

just run train step and see if agent learns any better

In [ ]:
from IPython.display import clear_output
from pandas import DataFrame
from tqdm import trange


moving_average = lambda x, **kw: DataFrame({'x': np.asarray(x)}).x.ewm(**kw).mean().values

rewards_history = []
grad_norm_history = []
entropy_history = []
loss_history = []

In [ ]:
for i in range(25000):
    memory = list(pool.prev_memory_states)
    rollout_obs, rollout_actions, rollout_rewards, rollout_mask = pool.interact(20)
    loss, grad_norm, entropy = train_on_rollout(
        rollout_obs, rollout_actions, rollout_rewards, rollout_mask, memory
    )
    grad_norm_history.append(grad_norm)
    entropy_history.append(entropy)
    loss_history.append(loss)
    if i % 100 == 0:
        rewards_history.append(np.mean(evaluate(agent, env, n_games=1)))
        clear_output(True)

        plt.figure(figsize=[16, 9])
        plt.subplot(2, 2, 1)
        plt.title("Mean reward")
        plt.plot(rewards_history, label='rewards')
        plt.plot(moving_average(np.array(rewards_history), span=10), label='rewards ewma@10')
        plt.legend()

        plt.subplot(2, 2, 2)
        plt.title("Grad norm history (smoothened)")
        plt.plot(moving_average(np.array(grad_norm_history), span=100), label='grad norm ewma@100')
        plt.grid()

        plt.subplot(2, 2, 3)
        plt.title("entropy (smoothened)")
        plt.plot(moving_average(np.array(entropy_history), span=100), label='entropy ewma@100')
        plt.grid()

        plt.subplot(2, 2, 4)
        plt.title("loss (smoothened)")
        plt.plot(np.array(loss_history), label='loss raw')
        plt.plot(moving_average(np.array(loss_history), span=10), label='loss ewma@10')
        plt.grid()

        plt.show()

        if rewards_history[-1] >= 10000:
            print("Your agent has just passed the minimum homework threshold")
            break

Relax and grab some refreshments while your agent is locked in an infinite loop of violence and death.

__How to interpret plots:__

The session reward is the easy thing: it should in general go up over time, but it's okay if it fluctuates ~~like crazy~~. It's also OK if it reward doesn't increase substantially before some 10k initial steps. However, if reward reaches zero and doesn't seem to get up over 2-3 evaluations, there's something wrong happening.


Since we use a policy-based method, we also keep track of __policy entropy__ - the same one you used as a regularizer. The only important thing about it is that your entropy shouldn't drop too low (`< 0.1`) before your agent gets the yellow belt. Or at least it can drop there, but _it shouldn't stay there for long_.

If it does, the culprit is likely:
* Some bug in entropy computation. Remember that it is $ - \sum p(a_i) \cdot log p(a_i) $
* Your agent architecture converges too fast. Increase entropy coefficient in actor loss. 
* Gradient explosion - just [clip gradients](https://stackoverflow.com/a/43486487) and maybe use a smaller network
* Us. Or TF developers. Or aliens. Or lizardfolk. Contact us on forums before it's too late!

If you're debugging, just run `logits, values = agent.step(batch_states)` and manually look into logits and values. This will reveal the problem 9 times out of 10: you'll likely see some NaNs or insanely large numbers or zeros. Try to catch the moment when this happens for the first time and investigate from there.

### "Final" evaluation

In [ ]:
with gym.wrappers.Monitor(env, directory="videos", force=True) as env_monitor:
    final_rewards = evaluate(agent, env_monitor, n_games=3)

print("Final mean reward", np.mean(final_rewards))

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from base64 import b64encode
from pathlib import Path

from IPython.display import HTML


video_paths = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[-1]  # You can also try other indices

if 'google.colab' in sys.modules:
    # https://stackoverflow.com/a/57378660/1214547
    with video_path.open('rb') as fp:
        mp4 = fp.read()
    data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
else:
    data_url = str(video_path)

HTML(
    """
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(
        data_url
    )
)